[![查看文章](https://img.shields.io/badge/查看%20文章-blue)](https://www.mongodb.com/developer/products/atlas/advanced-rag-langchain-mongodb/)

# 使用MongoDB和LangChain为您的RAG应用程序添加语义缓存和内存

在这个笔记本中，我们将看到如何在您的RAG应用程序中使用新的MongoDBCache和MongoDBChatMessageHistory。

## 步骤 1：安装所需的库

- **datasets**：Python 库，用于访问 Hugging Face Hub 上可用的数据集

- **langchain**：LangChain 的 Python 工具包

- **langchain-mongodb**：用于在 LangChain 中将 MongoDB 作为向量存储、语义缓存、聊天记录存储等的 Python 包

- **langchain-openai**：用于在 LangChain 中使用 OpenAI 模型的 Python 包

- **pymongo**：用于 MongoDB 的 Python 工具包

- **pandas**：用于数据分析、探索和操作的 Python 库

In [1]:
# 安装必要的库
! pip install -qU datasets langchain langchain-mongodb langchain-openai pymongo pandas

## 步骤2：设置前提条件

* 设置MongoDB连接字符串。按照[这里](https://www.mongodb.com/docs/manual/reference/connection-string/)的步骤从Atlas UI获取连接字符串。

* 设置OpenAI API密钥。获取API密钥的步骤如[这里](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key)所述。

In [2]:
# 导入 getpass 模块，用于获取用户输入的密码而不显示在屏幕上
import getpass

In [3]:
MONGODB_URI = getpass.getpass("请输入您的MongoDB连接字符串：")

Enter your MongoDB connection string:········


In [4]:
# 导入getpass模块
import getpass
# 提示用户输入OpenAI API密钥，并将其存储在OPENAI_API_KEY变量中
OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key:")

Enter your OpenAI API key:········


In [5]:
# 可选-- 如果您想启用Langsmith-- 适用于调试
import os

# 设置环境变量"LANGCHAIN_TRACING_V2"为"true"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# 设置环境变量"LANGCHAIN_API_KEY"为用户输入的密码，需要导入getpass模块
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

········


## 步骤 3：下载数据集

我们将使用 MongoDB 的 [embedded_movies](https://huggingface.co/datasets/MongoDB/embedded_movies) 数据集。

In [6]:
import pandas as pd
from datasets import load_dataset

In [ ]:
# 确保在开发环境中有一个HF_TOKEN：
# 访问令牌可以在Hugging Face平台上创建或复制（https://huggingface.co/docs/hub/en/security-tokens）

# 从Hugging Face加载MongoDB的embedded_movies数据集
# https://huggingface.co/datasets/MongoDB/airbnb_embeddings

data = load_dataset("MongoDB/embedded_movies")

In [8]:
# 导入 pandas 库
import pandas as pd

# 使用 data 字典中的 "train" 键创建一个 DataFrame 对象
df = pd.DataFrame(data["train"])


## 第四步：数据分析

确保数据集的长度符合我们的预期，删除空值等。

In [10]:
# 预览数据的内容
df.head(1)

,fullplot,type,plot_embedding,num_mflix_comments,runtime,writers,imdb,countries,rated,plot,title,languages,metacritic,directors,awards,genres,poster,cast
0,Young Pauline is left a lot of money when her ...,movie,"[0.00072939653, -0.026834568, 0.013515796, -0....",0,199.0,"[Charles W. Goddard (screenplay), Basil Dickey...","{'id': 4465, 'rating': 7.6, 'votes': 744}",[USA],None,Young Pauline is left a lot of money when her ...,The Perils of Pauline,[English],NaN,"[Louis J. Gasnier, Donald MacKenzie]","{'nominations': 0, 'text': '1 win.', 'wins': 1}",[Action],https://m.media-amazon.com/images/M/MV5BMzgxOD...,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar..."


In [11]:
# 只保留fullplot字段不为空的记录
df = df[df["fullplot"].notna()]

In [12]:
# 将嵌入字段重命名为"embedding" -- LangChain要求的必要步骤
df.rename(columns={"plot_embedding": "embedding"}, inplace=True)

## 步骤 5：使用 MongoDB 作为向量存储创建一个简单的 RAG 链

在这一步中，我们将使用 MongoDB 作为向量存储来创建一个简单的 RAG 链。

In [13]:
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient

# 初始化MongoDB的Python客户端
client = MongoClient(MONGODB_URI, appname="devrel.content.python")

DB_NAME = "langchain_chatbot"  # 数据库名称
COLLECTION_NAME = "data"  # 集合名称
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"  # 矢量搜索索引名称
collection = client[DB_NAME][COLLECTION_NAME]  # 获取指定数据库和集合的集合对象

In [14]:
# 删除集合中的所有现有记录
collection.delete_many({})

DeleteResult({'n': 1000, 'electionId': ObjectId('7fffffff00000000000000f6'), 'opTime': {'ts': Timestamp(1710523288, 1033), 't': 246}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1710523288, 1042), 'signature': {'hash': b"i\xa8\xe9'\x1ed\xf2u\xf3L\xff\xb1\xf5\xbfA\x90\xabJ\x12\x83", 'keyId': 7299545392000008318}}, 'operationTime': Timestamp(1710523288, 1033)}, acknowledged=True)

In [16]:
# 数据导入
# 将DataFrame转换为字典形式的记录
records = df.to_dict("records")
# 将记录插入到MongoDB集合中
collection.insert_many(records)

print("数据导入到MongoDB完成")

Data ingestion into MongoDB completed


In [18]:
from langchain_openai import OpenAIEmbeddings

# 使用text-embedding-ada-002模型，因为这是用于在电影数据集中创建嵌入的模型
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY, model="text-embedding-ada-002"
)

In [19]:
# 创建向量存储
# 从连接字符串创建MongoDBAtlasVectorSearch对象
vector_store = MongoDBAtlasVectorSearch.from_connection_string(
    connection_string=MONGODB_URI,  # 使用给定的连接字符串连接到MongoDB
    namespace=DB_NAME + "." + COLLECTION_NAME,  # 设置命名空间为给定的数据库名和集合名
    embedding=embeddings,  # 设置嵌入向量
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,  # 设置索引名称
    text_key="fullplot",  # 设置文本键为"fullplot"
)

In [49]:
# 在RAG链中使用MongoDB向量存储作为检索器
# 将向量存储转换为检索器
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [25]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# 使用检索器生成上下文，并将用户问题传递给检索器
retrieve = {
    "context": retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])),
    "question": RunnablePassthrough(),
}
template = """根据以下上下文回答问题：
{context}

问题：{question}
"""
# 定义聊天提示
prompt = ChatPromptTemplate.from_template(template)
# 定义用于聊天完成的模型
model = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
# 将输出解析为字符串
parse_output = StrOutputParser()

# Naive RAG链
naive_rag_chain = retrieve | prompt | model | parse_output

In [26]:
naive_rag_chain.invoke("当感到悲伤时，最好看哪部电影？")  # 调用naive_rag_chain模型，传入问题作为参数，返回回答

'Once a Thief'

## 步骤 6：使用聊天记录创建 RAG 链

In [27]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_mongodb.chat_message_histories import MongoDBChatMessageHistory

In [29]:
# 定义一个函数，函数名为get_session_history，参数为session_id，返回类型为MongoDBChatMessageHistory
def get_session_history(session_id: str) -> MongoDBChatMessageHistory:
    # 返回一个MongoDBChatMessageHistory对象，传入参数为MONGODB_URI、session_id、DB_NAME和"history"
    return MongoDBChatMessageHistory(
        MONGODB_URI, session_id, database_name=DB_NAME, collection_name="history"
    )

In [50]:
# 给定一个后续问题和历史记录，创建一个独立的问题
standalone_system_prompt = """
给定一个聊天历史记录和一个后续问题，将后续问题重新表达为一个独立的问题。\
如果需要，不要回答问题，只需重新表述问题，否则原样返回。\
只返回最终的独立问题。\
"""
standalone_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", standalone_system_prompt),  # 系统提示
        MessagesPlaceholder(variable_name="history"),  # 历史记录占位符
        ("human", "{question}"),  # 用户输入的问题
    ]
)

question_chain = standalone_question_prompt | model | parse_output  # 问题处理链

In [51]:
# 通过将问题链的输出（即独立的问题）传递给检索器来生成上下文
# 创建一个可运行的传递对象 retriever_chain
retriever_chain = RunnablePassthrough.assign(
    context=question_chain  # 将问题链的输出作为上下文
    | retriever  # 将上下文传递给检索器
    | (lambda docs: "\n\n".join([d.page_content for d in docs]))  # 将检索器返回的文档内容连接起来
)

In [55]:
# 创建一个包含上下文、历史记录和后续问题的提示
rag_system_prompt = """Answer the question based only on the following context: \
{context}
"""
# 创建一个聊天提示模板，包含系统提示、历史记录占位符和人类问题
rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rag_system_prompt),  # 系统提示
        MessagesPlaceholder(variable_name="history"),  # 历史记录占位符
        ("human", "{question}"),  # 人类问题
    ]
)

In [56]:
# RAG链
# 定义一个变量rag_chain，表示RAG链的组合
# retriever_chain表示检索器链
# rag_prompt表示RAG提示器
# model表示模型
# parse_output表示解析输出
rag_chain = retriever_chain | rag_prompt | model | parse_output

In [57]:
# RAG链与历史记录
# 创建一个带有历史记录的RunnableWithMessageHistory对象
with_message_history = RunnableWithMessageHistory(
    rag_chain,  # RAG链对象
    get_session_history,  # 获取会话历史记录的函数
    input_messages_key="question",  # 输入消息的键名
    history_messages_key="history",  # 历史消息的键名
)

# 调用带有历史记录的RAG链
with_message_history.invoke(
    {"question": "What is the best movie to watch when sad?"},  # 输入的问题
    {"configurable": {"session_id": "1"}},  # 可配置的参数，此处为会话ID
)

'The best movie to watch when feeling down could be "Last Action Hero." It\'s a fun and action-packed film that blends reality and fantasy, offering an escape from the real world and providing an entertaining distraction.'

In [58]:
# 使用 with_message_history 调用函数，并传入参数
with_message_history.invoke(
    {
        "question": "Hmmm..I don't want to watch that one. Can you suggest something else?"
    },
    {"configurable": {"session_id": "1"}},
)

'I apologize for the confusion. Another movie that might lift your spirits when you\'re feeling sad is "Smilla\'s Sense of Snow." It\'s a mystery thriller that could engage your mind and distract you from your sadness with its intriguing plot and suspenseful storyline.'

In [59]:


# 调用 with_message_history 模块的 invoke 函数
# 传入两个参数：一个是包含问题的字典，另一个是包含配置信息的字典
with_message_history.invoke(
    {"question": "How about something more light?"},
    {"configurable": {"session_id": "1"}},
)

'For a lighter movie option, you might enjoy "Cousins." It\'s a comedy film set in Barcelona with action and humor, offering a fun and entertaining escape from reality. The storyline is engaging and filled with comedic moments that could help lift your spirits.'

## 步骤 7：使用语义缓存获得更快的响应

**注意：** 语义缓存仅缓存到LLM的输入。在检索链中使用时，请记住检索到的文档可能会在运行之间发生变化，导致语义上相似的查询缓存未命中。

In [61]:
from langchain_core.globals import set_llm_cache
from langchain_mongodb.cache import MongoDBAtlasSemanticCache

set_llm_cache(
    MongoDBAtlasSemanticCache(
        connection_string=MONGODB_URI,
        embedding=embeddings,
        collection_name="semantic_cache",
        database_name=DB_NAME,
        index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
        wait_until_ready=True,  # Optional, waits until the cache is ready to be used
    )
)

In [62]:
%%time
# 计算代码运行时间
naive_rag_chain.invoke("What is the best movie to watch when sad?")
# 调用naive_rag_chain的invoke方法，传入"What is the best movie to watch when sad?"作为参数，获取结果

CPU times: user 87.8 ms, sys: 670 µs, total: 88.5 ms
Wall time: 1.24 s


'Once a Thief'

In [63]:
%%time
# 计算代码执行时间

naive_rag_chain.invoke("What is the best movie to watch when sad?")
# 调用naive_rag_chain的invoke方法，传入"What is the best movie to watch when sad?"作为参数。该方法用于生成回答给定问题的响应。

CPU times: user 43.5 ms, sys: 4.16 ms, total: 47.7 ms
Wall time: 255 ms


'Once a Thief'

In [64]:
# 计算代码执行时间
%%time
# 调用naive_rag_chain模型，传入问题"Which movie do I watch when sad?"
naive_rag_chain.invoke("Which movie do I watch when sad?")

CPU times: user 115 ms, sys: 171 µs, total: 115 ms
Wall time: 1.38 s


'I would recommend watching "Last Action Hero" when sad, as it is a fun and action-packed film that can help lift your spirits.'